## Feature extraction

A document is nothing but a group (more than one) of tokens.Every supervised machine learning algorithm requires each and every textual document to be represented in the form of a vector to start training on these docs,this is done through Vector Space Modelling (VSM).

Differ from traditional **one-hot** vector, VSM can largely be implemented via two unique and poles apart techniques:

* count-based method: BOW and tf-idf
* predication-based method: word2vec and Doc2vec

## BOW

In [3]:
CORPUS = [
'the sky is blue',
'sky is blue and sky is beautiful',
'the beautiful sky is so blue',
'i love blue cheese'
]

In [4]:
new_doc = ['loving this blue sky today']

import pandas as pd

def display_features(features, feature_names):
    df = pd.DataFrame(data=features,
                      columns=feature_names)
    print df


from feature_extractors import bow_extractor    
    
bow_vectorizer, bow_features = bow_extractor(CORPUS)
features = bow_features.todense()
feature_names = bow_vectorizer.get_feature_names()
display_features(features, feature_names)

   and  beautiful  blue  cheese  is  love  sky  so  the
0    0          0     1       0   1     0    1   0    1
1    1          1     1       0   2     0    2   0    0
2    0          1     1       0   1     0    1   1    1
3    0          0     1       1   0     1    0   0    0


In [5]:
new_doc_features = bow_vectorizer.transform(new_doc)
new_doc_features = new_doc_features.todense()
display_features(new_doc_features, feature_names)

   and  beautiful  blue  cheese  is  love  sky  so  the
0    0          0     1       0   0     0    1   0    0


## TF-IDF
<img src="tf.png" width="200">
<img src="idf.png" width="200">

In [6]:
import numpy as np
from feature_extractors import tfidf_transformer
feature_names = bow_vectorizer.get_feature_names()
    
tfidf_trans, tdidf_features = tfidf_transformer(bow_features)
features = np.round(tdidf_features.todense(), 2)
display_features(features, feature_names)

    and  beautiful  blue  cheese    is  love   sky    so   the
0  0.00       0.00  0.40    0.00  0.49  0.00  0.49  0.00  0.60
1  0.44       0.35  0.23    0.00  0.56  0.00  0.56  0.00  0.00
2  0.00       0.43  0.29    0.00  0.35  0.00  0.35  0.55  0.43
3  0.00       0.00  0.35    0.66  0.00  0.66  0.00  0.00  0.00


## Word2Vec
This algorithm transfers word to vector using word embedding solution. It has two models: **Skip-Gram** and **CBOW**.

In [8]:
# using existing word2vec of gensim 
import gensim
import nltk

TOKENIZED_CORPUS = [nltk.word_tokenize(sentence) for sentence in CORPUS]
tokenized_new_doc = [nltk.word_tokenize(sentence) for sentence in new_doc]

model = gensim.models.Word2Vec(TOKENIZED_CORPUS,
                               size=10,
                               window=10,
                               min_count=2,
                               sample=1e-3)

print model

y1 = model.wv.similarity(u"sky", u"beautiful")
print "the smilarity of [sky] and [beautiful] is:", y1
print"-----\n"

y2 = model.wv.most_similar("sky", topn=2)  # 2个最相关的
print u"the words related to [sky] are:"
for item in y2:
    print item[0], item[1]
print"-----\n"
 
y4 =model.wv.doesnt_match(u"sky blue love".split())
print "the word doesn't match:"
print y4
print"-----\n"

print "save the model"
model.wv.save("sample.model")

Word2Vec(vocab=5, size=10, alpha=0.025)
the smilarity of [sky] and [beautiful] is: -0.19884284
-----

the words related to [sky] are:
the 0.0992874801159
is -0.13096781075
-----

the word doesn't match:
blue
-----

save the model


### CBOW
<img src="CBOW.png" width="400">
(1) <img src="CBOW1.png" width="400">
(2) <img src="CBOW2.png" width="400">
(3) <img src="CBOW3.png" width="400">
(4) <img src="CBOW4.png" width="400">
(5) <img src="CBOW5.png" width="400">

### Skip-Gram
<img src="skip-gram.png" width="400">
(1) <img src="skip-gram1.png" width="400">
(2) <img src="skip-gram2.png" width="400">
(3) <img src="skip-gram3.png" width="400">

In [10]:
import numpy as np
import collections
class word2vec():
    def __init__(self):
        self.n = settings['n']
        self.lr = settings['learning_rate']
        self.epochs = settings['epochs']
        self.window = settings['window_size']

    def generate_training_data(self, settings, corpus):
        # Find unique word counts using dictonary
        word_counts = collections.defaultdict(int)
        for row in corpus:
            for word in row:
                word_counts[word] += 1
        ## How many unique words in vocab? 9
        self.v_count = len(word_counts.keys())
        # Generate Lookup Dictionaries (vocab)
        self.words_list = list(word_counts.keys())
        # Generate word:index
        self.word_index = dict((word, i) for i, word in enumerate(self.words_list))
        # Generate index:word
        self.index_word = dict((i, word) for i, word in enumerate(self.words_list))

        training_data = []
        # Cycle through each sentence in corpus
        for sentence in corpus:
            sent_len = len(sentence)
            # Cycle through each word in sentence
            for i, word in enumerate(sentence):
                # Convert target word to one-hot
                w_target = self.word2onehot(sentence[i])
                # Cycle through context window
                w_context = []
                # Note: window_size 2 will have range of 5 values
                for j in range(i - self.window, i + self.window+1):
                    # Criteria for context word
                    # 1. Target word cannot be context word (j != i)
                    # 2. Index must be greater or equal than 0 (j >= 0) - if not list index out of range
                    # 3. Index must be less or equal than length of sentence (j <= sent_len-1) - if not list index out of range
                    if j != i and j <= sent_len-1 and j >= 0:
                    # Append the one-hot representation of word to w_context
                        w_context.append(self.word2onehot(sentence[j]))
                        # print(sentence[i], sentence[j])
                        # training_data contains a one-hot representation of the target word and context words
                    training_data.append([w_target, w_context])
    
        return np.array(training_data)

    def word2onehot(self, word):
        # word_vec - initialise a blank vector
        word_vec = [0 for i in range(0, self.v_count)] # Alternative - np.zeros(self.v_count)
        # Get ID of word from word_index
        word_index = self.word_index[word]
        # Change value from 0 to 1 according to ID of the word
        word_vec[word_index] = 1
        return word_vec

    def train(self, training_data):
        
        self.w1 = np.random.uniform(-1, 1, (self.v_count, self.n))
        self.w2 = np.random.uniform(-1, 1, (self.n, self.v_count))
        ##Removed##
        # Cycle through each epoch
        for i in range(self.epochs):
            self.loss = 0
            for w_t, w_c in training_data:
            ##Removed##
            # Calculate error
            # 1. For a target word, calculate difference between y_pred and each of the context words
            # 2. Sum up the differences using np.sum to give us the error for this particular target word
                y_pred, h, u = self.forward_pass(w_t)
                EI = np.sum([np.subtract(y_pred, word) for word in w_c], axis=0)
            # Backpropagation
            # We use SGD to backpropagate errors - calculate loss on the output layer
                self.backprop(EI, h, w_t)
        
            # Calculate loss
            # There are 2 parts to the loss function
            # Part 1: -ve sum of all the output +
            # Part 2: length of context words * log of sum for all elements (exponential-ed) in the output layer before softmax (u)
            # Note: word.index(1) returns the index in the context word vector with value 1
            # Note: u[word.index(1)] returns the value of the output layer before softmax
                self.loss += -np.sum([u[word.index(1)] for word in w_c]) + len(w_c) * np.log(np.sum(np.exp(u)))
            
            print('Epoch:', i, "Loss:", self.loss)

    def backprop(self, e, h, x):
        # https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.outer.html
        # Column vector EI represents row-wise sum of prediction errors across each context word for the current center word
        # Going backwards, we need to take derivative of E with respect of w2
        # h - shape 10x1, e - shape 9x1, dl_dw2 - shape 10x9
        dl_dw2 = np.outer(h, e)
        # x - shape 1x8, w2 - 5x8, e.T - 8x1
        # x - 1x8, np.dot() - 5x1, dl_dw1 - 8x5
        dl_dw1 = np.outer(x, np.dot(self.w2, e.T))
        # Update weights
        self.w1 = self.w1 - (self.lr * dl_dw1)
        self.w2 = self.w2 - (self.lr * dl_dw2)
        
    def forward_pass(self, x):
        # x is one-hot vector for target word, shape - 9x1
        # Run through first matrix (w1) to get hidden layer - 10x9 dot 9x1 gives us 10x1
        h = np.dot(self.w1.T, x)
        # Dot product hidden layer with second matrix (w2) - 9x10 dot 10x1 gives us 9x1
        u = np.dot(self.w2.T, h)
        # Run 1x9 through softmax to force each element to range of [0, 1] - 1x8
        y_c = self.softmax(u)
        return y_c, h, u
    
    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    def word_vec(self, word):
        w_index = self.word_index[word]
        v_w = self.w1[w_index]
        return v_w

    def vec_sim(self, word, top_n):
        v_w1 = self.word_vec(word)
        word_sim = {}
    
        for i in range(self.v_count):
            # Find the similary score for each word in vocab
            v_w2 = self.w1[i]
            theta_sum = np.dot(v_w1, v_w2)
            theta_den = np.linalg.norm(v_w1) * np.linalg.norm(v_w2)
            theta = theta_sum / theta_den
        
            word = self.index_word[i]
            word_sim[word] = theta
    
        words_sorted = sorted(word_sim.items(), key=lambda kv: kv[1], reverse=True)
    
        for word, sim in words_sorted[:top_n]:
            print(word, sim)
text = "natural language processing and machine learning is fun and exciting"

# Note the .lower() as upper and lowercase does not matter in our implementation
# [['natural', 'language', 'processing', 'and', 'machine', 'learning', 'is', 'fun', 'and', 'exciting']]
corpus = [[word.lower() for word in text.split()]]

settings = {
	'window_size': 2, # context window +- center word
	'n': 10,		# dimensions of word embeddings, also refer to size of hidden layer
	'epochs': 50,		# number of training epochs
	'learning_rate': 0.01# learning rate
}

# Initialise object
w2v = word2vec()
# Numpy ndarray with one-hot representation for [target_word, context_words]
training_data = w2v.generate_training_data(settings, corpus)
w2v.train(training_data)
vec = w2v.word_vec("machine")
w2v.vec_sim("machine", 3)

('Epoch:', 0, 'Loss:', 458.26127287819514)
('Epoch:', 1, 'Loss:', 413.6798383626999)
('Epoch:', 2, 'Loss:', 385.1572244201346)
('Epoch:', 3, 'Loss:', 365.01276864128084)
('Epoch:', 4, 'Loss:', 349.7955040809828)
('Epoch:', 5, 'Loss:', 337.75059351963404)
('Epoch:', 6, 'Loss:', 327.88592030771196)
('Epoch:', 7, 'Loss:', 319.5884010127762)
('Epoch:', 8, 'Loss:', 312.45484469758543)
('Epoch:', 9, 'Loss:', 306.2105603005523)
('Epoch:', 10, 'Loss:', 300.66450826206295)
('Epoch:', 11, 'Loss:', 295.6815736893177)
('Epoch:', 12, 'Loss:', 291.16463560752436)
('Epoch:', 13, 'Loss:', 287.04298093470135)
('Epoch:', 14, 'Loss:', 283.26484382733645)
('Epoch:', 15, 'Loss:', 279.79242867064136)
('Epoch:', 16, 'Loss:', 276.59823854943994)
('Epoch:', 17, 'Loss:', 273.6619970817738)
('Epoch:', 18, 'Loss:', 270.9678923049268)
('Epoch:', 19, 'Loss:', 268.5021991317503)
('Epoch:', 20, 'Loss:', 266.2514799959319)
('Epoch:', 21, 'Loss:', 264.2015220114629)
('Epoch:', 22, 'Loss:', 262.33701675959554)
('Epoch:'

###  Optimizing Computational Efficiency
* Hierarchical Softmax
<img src="Hierarchical Softmax.png" width="400">
* Negative Sampling
<img src="Negative Sampling.png" width="400">

## Doc2Vec
This algorithm transfers document to vector. It has two models:
* PV-DM(Paragraph Vector: A distributed memory model)
<img src="PVDM.png" width="400">
* DBOW(Distributed bag of words)
<img src="DBOW.png" width="400">

In [11]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")
model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
vector = model.infer_vector(["system", "response"])
print vector

[-0.00349501 -0.06090447 -0.07299673  0.04204015  0.01652598]


## Reference
* https://arxiv.org/pdf/1301.3781.pdf
* https://arxiv.org/pdf/1411.2738.pdf
* https://cs.stanford.edu/~quocle/paragraph_vector.pdf